# weather in Australia

In [ ]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
data = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data['Date']=data.Date.apply(lambda x:x[:4])
data['Date']=data['Date'].astype('int')

In [ ]:
data.isna().sum()

In [ ]:
data.corr()

In [ ]:
data2 = data.apply(lambda x:x.fillna(x.median()) if x.dtype=='float' else x.fillna(x.value_counts().index[0]))

In [ ]:
data2['RainToday'].value_counts()

# data visualization

In [ ]:
fig = px.scatter(data2, y="Evaporation", x="Humidity3pm",size="Rainfall",color="RainTomorrow",hover_name="Location",template='plotly_dark',log_y=True, size_max=40)

fig.update_traces(marker=dict(line=dict(width=1,color='white')))
fig.show()

In [ ]:
sb.histplot(y=data['Rainfall'],x=data['Date'])
plt.title('year wise rainfall')
plt.show()

In [ ]:
fig = px.pie(data2, values='Rainfall', names='WindGustDir')
fig.update_layout(title=' Rainfall vs wind gust direction')
fig.show()

In [ ]:
df = data2
fig = px.pie(df, values='Rainfall', names='Location')
fig.update_layout(title='rainfall vs location')
fig.show()

In [ ]:
fig = px.area(df, x="Humidity3pm", y="Rainfall", color="WindGustDir",line_group="Location")
fig.show()

In [ ]:
tomorrow = data[['Rainfall','Sunshine','Humidity9am','Humidity3pm','Cloud9am','Cloud3pm','RainToday','RainTomorrow']]

In [ ]:
print(tomorrow.isna().sum())

In [ ]:
tomorrow=tomorrow.apply(lambda x:x.fillna(x.median()) if x.dtype=='float' else x.fillna(x.value_counts().index[0]))

In [ ]:
print(tomorrow.isna().sum())

In [ ]:
tomorrow['RainToday'],_=pd.factorize(tomorrow.RainToday)
tomorrow['RainTomorrow'],_=pd.factorize(tomorrow.RainTomorrow)

In [ ]:
tomorrow.head()

In [ ]:
tomorrow.corr()

# model building

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV

In [ ]:
features=['Rainfall','Sunshine','Humidity9am','Humidity3pm','Cloud9am','Cloud3pm','RainToday']

In [ ]:
x=tomorrow[features]
y=tomorrow['RainTomorrow']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=5,test_size=0.3)

In [ ]:
param_grid = {
                 'n_estimators': [5, 10, 15, 20],
                 'max_depth': [2, 5, 7, 9]
             }
grid_clf = GridSearchCV(RandomForestClassifier(), param_grid, cv=10)
grid_clf.fit(x_train, y_train)

In [ ]:
grid_clf. best_estimator_

In [ ]:
grid_clf. best_params_

In [ ]:
grid_clf.cv_results_

In [ ]:
ran = RandomForestClassifier(max_depth=9, n_estimators=50, random_state=1)

In [ ]:
ran.fit(x_train,y_train)

In [ ]:
ran_pred = ran.predict(x_test)

In [ ]:
print(accuracy_score(y_test,ran_pred))

In [ ]:
from sklearn.model_selection import cross_val_score

clf = ran

scores = cross_val_score(clf, x,y, cv=10)
print('validation scores :\n',scores)

In [ ]:
print(classification_report(y_test,ran_pred))

## The model predicts with 84% accuracy